### Sub_hot_1 Sub_hot_4 Sub_xgb_1 gbm_2 gbm_3

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
train_data = pd.read_json('train_data.json',orient="index")
test_data = pd.read_json('test_data.json',orient='index')

In [3]:
#set index
train_data.reset_index(inplace = True)
train_data.rename(columns={'index':'ID'}, inplace=True)

In [4]:
test_data.reset_index(inplace = True)
test_data.rename(columns={'index':'ID'}, inplace=True)

In [5]:
train_data.head(2)

,ID,cities,dow,genres,segment,titles,tod
0,train-1,"gurgaon:55494,delhi:31892","1:3412,3:15878,2:1737,5:10975,4:20974,7:17820,...","Cricket:82379,Kabaddi:255,Reality:4751",neg,"Top Raids: Haryana vs Services SCB:103,Day 4: ...","10:26,13:331,12:323,20:21864,21:16233,17:7953,..."
1,train-10,"delhi:5862,nagar:8916,mumbai:1593","1:5745,3:3025,2:3346,5:123,4:3007,7:1108,6:10","Cricket:15640,Wildlife:730",neg,"Dhoni Quits Captaincy:148,Day 4: India Move in...","11:1661,10:384,20:401,21:798,22:221,16:525,19:..."


In [6]:
train_data.titles[5]

u'India vs Australia 2nd Test English:2836,SRH vs RCB:406,KKR vs KXIP:177,MI vs SRH:502,Dominant India Win Series in Style:427,Lyon Pulls Oz Back on Day 2:29,Kohli Leads the Way with 122:82,1st ODI: India Begin Kohli Era with Win:512,All You Need to Know About One-off Test:40,India vs Australia 3rd Test English:459,India vs England | 2nd ODI | English:1198,India vs Bangladesh Day 3 English:374,Ruthless India Stay Perfect at Home:700,India Begin Kohli Era with Win:1022,RCB vs DD:1768,India 1st Innings: Fall of Wickets:420,Day 3: India Finally Break Oz Spirit:317,Aus Turn the Tide to Humble India:14,Bodyguard:998,RPS vs DD:244,MS Dhoni: The Untold Story:8786,Kaabil:9,KXIP vs RPS:3370,Lyon Rips Out India with 5-for:16,India vs Bangladesh Day 1 English:2535,India vs Australia 1st Test English:1453,RCB vs MI:4432,India vs England 2nd T20I English:7720,Ind Win Thriller Keep Series Alive:947,India vs Australia 4th Test English:2244,India Win Kohli Creates History:147,Resilient India on Top af

In [7]:
test_data.head(2)

,ID,cities,dow,genres,titles,tod
0,test-1,"delhi:182,mumbai:702","2:182,4:701","Romance:182,Cricket:702","Axar's 21-ball 40 amidst KXIP ruins:72,Virat K...","13:182,15:701"
1,test-10,"pune:16,delhi:1100,navi mumbai:22736","1:4830,3:3078,2:8359,5:2946,4:4185,7:451","Romance:50,Action:3501,Mythology:20300","Besh Korechi Prem Korechi:0,Power:122,Siya Ke ...","20:6909,21:1520,22:662,23:3160,19:5711,1:121,0..."


In [8]:
#check data
print ('Train data has {} rows and {} columns'.format(train_data.shape[0],train_data.shape[1]))
print ('test_data data has {} rows and {} columns'.format(test_data.shape[0],test_data.shape[1]))

Train data has 200000 rows and 7 columns
test_data data has 100000 rows and 6 columns


In [9]:
#Encode Target Variable
train_data = train_data.replace({'segment':{'pos':1,'neg':0}})

In [10]:
#check target variable count
train_data['segment'].value_counts()/train_data.shape[0]

0    0.923725
1    0.076275
Name: segment, dtype: float64

In [11]:
train_data['g1'] = [re.sub(pattern='\:\d+',repl='',string=x) for x in train_data['genres']]

In [12]:
train_data.g1[0:5]

0    Cricket,Kabaddi,Reality
1           Cricket,Wildlife
2            LiveTV,Football
3           TalkShow,Cricket
4      Drama,Cricket,Reality
Name: g1, dtype: object

In [13]:
train_data['g1'] = train_data['g1'].apply(lambda x: x.split(','))

In [14]:
train_data['g2'] = [re.sub(pattern='\:\d+', repl='', string = x) for x in train_data['dow']]
train_data['g2'] = train_data['g2'].apply(lambda x: x.split(','))

In [15]:
train_data.g2[0:5]

0    [1, 3, 2, 5, 4, 7, 6]
1    [1, 3, 2, 5, 4, 7, 6]
2                      [3]
3          [1, 3, 5, 4, 7]
4          [1, 2, 4, 7, 6]
Name: g2, dtype: object

In [16]:
t1 = pd.Series(train_data['g1']).apply(frozenset).to_frame(name='t_genre')

In [17]:
t1[0:5]

,t_genre
0,"(Cricket, Kabaddi, Reality)"
1,"(Cricket, Wildlife)"
2,"(LiveTV, Football)"
3,"(TalkShow, Cricket)"
4,"(Drama, Cricket, Reality)"


In [18]:
t2 = pd.Series(train_data['g2']).apply(frozenset).to_frame(name='t_dow')

In [19]:
# using frozenset trick - might take few minutes to process
for t_genre in frozenset.union(*t1.t_genre):
    t1[t_genre] = t1.apply(lambda _: int(t_genre in _.t_genre), axis=1)

In [20]:
t1[0:3]

,t_genre,Travel,Boxing,Crime,Romance,LiveTV,Hockey,FormulaE,Comedy,Teen,...,Drama,Action,Athletics,Reality,Documentary,Swimming,Formula1,Family,Badminton,Sport
0,"(Cricket, Kabaddi, Reality)",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,"(Cricket, Wildlife)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(LiveTV, Football)",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
for t_dow in frozenset.union(*t2.t_dow):
    t2[t_dow] = t2.apply(lambda _: int(t_dow in _.t_dow), axis = 1)

In [22]:
train_data = pd.concat([train_data.reset_index(drop=True), t1], axis=1)
train_data = pd.concat([train_data.reset_index(drop=True), t2], axis=1)

In [23]:
test_data['g1'] = [re.sub(pattern='\:\d+',repl='',string=x) for x in test_data['genres']]
test_data['g1'] = test_data['g1'].apply(lambda x: x.split(','))

test_data['g2'] = [re.sub(pattern='\:\d+', repl='', string = x) for x in test_data['dow']]
test_data['g2'] = test_data['g2'].apply(lambda x: x.split(','))

t1_te = pd.Series(test_data['g1']).apply(frozenset).to_frame(name='t_genre')
t2_te = pd.Series(test_data['g2']).apply(frozenset).to_frame(name='t_dow')

In [24]:
for t_genre in frozenset.union(*t1_te.t_genre):
    t1_te[t_genre] = t1_te.apply(lambda _: int(t_genre in _.t_genre), axis=1)

In [25]:
for t_dow in frozenset.union(*t2_te.t_dow):
    t2_te[t_dow] = t2_te.apply(lambda _: int(t_dow in _.t_dow), axis = 1)

In [26]:
test_data = pd.concat([test_data.reset_index(drop=True), t1_te], axis=1)
test_data = pd.concat([test_data.reset_index(drop=True), t2_te], axis=1)

In [27]:
#the rows aren't list exactly. They are object, so we convert them to list and extract the watch time
w1 = train_data['titles']
w1 = w1.str.split(',')

#create a nested list of numbers
main = []
for i in np.arange(train_data.shape[0]):
    d1 = w1[i]
    nest = []
    nest = [re.sub(pattern = '.*\:', repl=' ', string= d1[k]) for k in list(np.arange(len(d1)))]
    main.append(nest)

In [28]:
main[2]

[u' 13', u' 4129']

In [29]:
blanks = []
for i in np.arange(len(main)):
    if '' in main[i]:
        print "{} blanks found".format(len(blanks))
        blanks.append(i)
        
#replacing blanks with 0
for i in blanks:
    main[i] = [x.replace('','0') for x in main[i]]
    
#converting string to integers
main = [[int(y) for y in x] for x in main]

#adding the watch time
tosum = []
for i in np.arange(len(main)):
    s = sum(main[i])
    tosum.append(s)

0 blanks found
1 blanks found
2 blanks found
3 blanks found
4 blanks found
5 blanks found
6 blanks found
7 blanks found
8 blanks found
9 blanks found
10 blanks found
11 blanks found
12 blanks found
13 blanks found
14 blanks found
15 blanks found
16 blanks found
17 blanks found
18 blanks found
19 blanks found
20 blanks found
21 blanks found
22 blanks found
23 blanks found
24 blanks found
25 blanks found
26 blanks found
27 blanks found
28 blanks found
29 blanks found
30 blanks found
31 blanks found
32 blanks found
33 blanks found
34 blanks found
35 blanks found
36 blanks found
37 blanks found
38 blanks found
39 blanks found
40 blanks found
41 blanks found
42 blanks found
43 blanks found
44 blanks found
45 blanks found


In [30]:
tosum[0]

87363

In [31]:
train_data['title_sum'] = tosum

In [32]:
#making changes in test data
w1_te = test_data['titles']
w1_te = w1_te.str.split(',')

In [33]:
main_te = []
for i in np.arange(test_data.shape[0]):
    d1 = w1_te[i]
    nest = []
    nest = [re.sub(pattern = '.*\:', repl=' ', string= d1[k]) for k in list(np.arange(len(d1)))]
    main_te.append(nest)

In [34]:
blanks_te = []
for i in np.arange(len(main_te)):
    if '' in main_te[i]:
        print "{} blanks found".format(len(blanks_te))
        blanks_te.append(i)
        
#replacing blanks with 0
for i in blanks_te:
    main_te[i] = [x.replace('','0') for x in main_te[i]]
    
#converting string to integers
main_te = [[int(y) for y in x] for x in main_te]

#adding the watch time
tosum_te = []
for i in np.arange(len(main_te)):
    s = sum(main_te[i])
    tosum_te.append(s)

0 blanks found
1 blanks found
2 blanks found
3 blanks found
4 blanks found
5 blanks found
6 blanks found
7 blanks found
8 blanks found
9 blanks found
10 blanks found
11 blanks found


In [35]:
test_data['title_sum'] = tosum_te

In [36]:
#count variables
def wcount(p):
    return p.count(',')+1

In [37]:
train_data['title_count'] = train_data['titles'].map(wcount)
train_data['genres_count'] = train_data['genres'].map(wcount)
train_data['cities_count'] = train_data['cities'].map(wcount)
train_data['tod_count'] = train_data['tod'].map(wcount)
train_data['dow_count'] = train_data['dow'].map(wcount)


test_data['title_count'] = test_data['titles'].map(wcount)
test_data['genres_count'] = test_data['genres'].map(wcount)
test_data['cities_count'] = test_data['cities'].map(wcount)
test_data['tod_count'] = test_data['tod'].map(wcount)
test_data['dow_count'] = test_data['dow'].map(wcount)

In [38]:
train_data.tod[0]

u'10:26,13:331,12:323,20:21864,21:16233,17:7953,16:1725,19:11505,18:6017,22:8894,2:2363,14:2196,15:2901,23:5046'

In [39]:
test_id = test_data['ID']
train_data.drop(['ID','cities','dow','genres','titles','tod','g1','g2','t_genre','t_dow'], inplace=True, axis=1)
test_data.drop(['ID','cities','dow','genres','titles','tod','g1','g2','t_genre','t_dow'], inplace=True, axis=1)

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [41]:
target = train_data['segment']
train_data.drop('segment',axis=1, inplace=True)

In [49]:
train_data.head(2)

,Travel,Boxing,Crime,Romance,LiveTV,Hockey,FormulaE,Comedy,Teen,Cricket,...,5,4,7,6,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
0,0,0,0,0,0,0,0,0,0,1,...,1,1,1,1,87363,60,3,2,14,7
1,0,0,0,0,0,0,0,0,0,1,...,1,1,1,1,16356,70,2,3,16,7


In [50]:
train_data.columns

Index([      u'Travel',       u'Boxing',        u'Crime',      u'Romance',
             u'LiveTV',       u'Hockey',     u'FormulaE',       u'Comedy',
               u'Teen',      u'Cricket',    u'Mythology',           u'NA',
             u'Horror',     u'Football',       u'Awards',      u'Science',
             u'Tennis',     u'Thriller',      u'Kabaddi',     u'Wildlife',
               u'Kids',    u'IndiaVsSa',     u'TalkShow', u'Table Tennis',
         u'Volleyball',        u'Drama',       u'Action',    u'Athletics',
            u'Reality',  u'Documentary',     u'Swimming',     u'Formula1',
             u'Family',    u'Badminton',        u'Sport',            u'1',
                  u'3',            u'2',            u'5',            u'4',
                  u'7',            u'6',    u'title_sum',  u'title_count',
       u'genres_count', u'cities_count',    u'tod_count',    u'dow_count'],
      dtype='object')

In [51]:
test_data.columns

Index([      u'Travel',      u'Kabaddi',      u'Reality',      u'Romance',
             u'LiveTV',       u'Hockey',     u'FormulaE',       u'Comedy',
               u'Teen',      u'Cricket',           u'NA',       u'Horror',
           u'Football',       u'Awards',      u'Science',       u'Tennis',
           u'Thriller',       u'Boxing',     u'Wildlife',         u'Kids',
          u'IndiaVsSa',     u'TalkShow', u'Table Tennis',   u'Volleyball',
              u'Drama',       u'Action',    u'Athletics',    u'Mythology',
        u'Documentary',     u'Swimming',     u'Formula1',       u'Family',
          u'Badminton',        u'Crime',        u'Sport',            u'1',
                  u'3',            u'2',            u'5',            u'4',
                  u'7',            u'6',    u'title_sum',  u'title_count',
       u'genres_count', u'cities_count',    u'tod_count',    u'dow_count'],
      dtype='object')

In [42]:
train_data["Sports"] = train_data.Boxing + train_data.Hockey + train_data.FormulaE + train_data.Cricket + train_data.Football + train_data.Tennis + train_data.Kabaddi + train_data.IndiaVsSa + train_data["Table Tennis"] + train_data.Volleyball + train_data.Athletics + train_data.Swimming + train_data.Formula1 + train_data.Badminton + train_data.Sport

In [43]:
test_data["Sports"] = test_data.Boxing + test_data.Hockey + test_data.FormulaE + test_data.Cricket + test_data.Football + test_data.Tennis + test_data.Kabaddi + test_data.IndiaVsSa + test_data["Table Tennis"] + test_data.Volleyball + test_data.Athletics + test_data.Swimming + test_data.Formula1 + test_data.Badminton + test_data.Sport

In [44]:
tt = train_data[["Travel","Reality","Romance","LiveTV","Comedy","Teen","NA","Horror","Awards","Science","Thriller",
               "Wildlife","Kids","TalkShow","Drama","Action","Mythology","Documentary","Family","Crime","Sports",
                "1","2","3","4","5","6","7","title_sum","title_count","genres_count","cities_count","tod_count",
                "dow_count"]]

In [45]:
tt.head(2)

,Travel,Reality,Romance,LiveTV,Comedy,Teen,NA,Horror,Awards,Science,...,4,5,6,7,title_sum,title_count,genres_count,cities_count,tod_count,dow_count
0,0,1,0,0,0,0,0,0,0,0,...,1,1,1,1,87363,60,3,2,14,7
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,16356,70,2,3,16,7


In [46]:
tttest = test_data[["Travel","Reality","Romance","LiveTV","Comedy","Teen","NA","Horror","Awards","Science","Thriller",
               "Wildlife","Kids","TalkShow","Drama","Action","Mythology","Documentary","Family","Crime","Sports",
                "1","2","3","4","5","6","7","title_sum","title_count","genres_count","cities_count","tod_count",
                "dow_count"]]

In [47]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.04).fit(tt, target)
pred = gbm.predict_proba(tttest)

In [88]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=rf_pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_xgboost_1.csv", index=False)

In [48]:
gbm = xgb.XGBClassifier(
    learning_rate = 0.05,
    n_estimators= 300,
    max_depth= 4,
    min_child_weight= 2,
    gamma=1,
    #gamma=0.9,                        
    subsample=0.8,
    colsample_bytree=0.8,
    #objective= 'binary:logistic',
    nthread= -1,
    scale_pos_weight=1).fit(tt, target)
pred_2 = gbm.predict_proba(tttest)

In [105]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=predictions_2[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("gbm_2.csv", index=False)

In [52]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=pred_3[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("pred_3_gbm_3.csv", index=False)

In [53]:
p = 0.35*pred+0.65*pred_2

In [54]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=p[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("0.65pred_2_0.35pred.csv", index=False)

In [57]:
p2 = 0.45*pred+0.55*pred_2

In [58]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=p2[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("0.55pred_2_0.45pred.csv", index=False)

In [61]:
p3 = 0.5*pred+0.5*pred_2

In [62]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=p3[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("0.5pred_2_0.5pred.csv", index=False)